# Spectrum

In [22]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [23]:
catalog = pd.read_csv("markeddata/gevtev_simbadclasses.txt", sep='\t', index_col=0)

In [24]:
catalog

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad,marked
0,19,b'0FGL J0007.4+7303',b'1FGL J0007.0+7303 ',b'1FHL J0007.3+7303 ',b'2FGL J0007.0+7303 ',b'LAT PSR J0007+7303 ',b' ',b'1AGL J0006+7311',b' ',b'EGR J0008+7308 ',...,b'rC5JCj',227,b'TeV J0006+729',1,b'TeV J0006+7259',b'gal',[],[],0,0
1,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'70hRXZ',213,b'TeV J0013-188',2,b'TeV J0013-1853',b'egal',[],[],0,0
2,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'xCyAjF',209,b'TeV J0025+641',3,b'TeV J0025+6410',b'gal',[],[],0,0
3,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'PKJZqs',239,b'TeV J0033-193',4,b'TeV J0033-1921',b'egal',[],[],0,0
4,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'eNYfrm',229,b'TeV J0035+598',5,b'TeV J0035+5947',b'egal',[],[],0,0
5,99,b' ',b'1FGL J0047.3-2512 ',b' ',b'2FGL J0047.0-2516 ',b'NGC 253 ',b' ',b' ',b' ',b' ',...,b'sK2rQU',195,b'TeV J0047-253',6,b'TeV J0047-2517',b'egal',[],[],0,0
6,193,b'0FGL J0136.6+3903',b'1FGL J0136.5+3905 ',b'1FHL J0136.5+3906 ',b'2FGL J0136.5+3905 ',b'B3 0133+388 ',b' ',b' ',b' ',b' ',...,b'7WhQrw',244,b'TeV J0136+391',7,b'TeV J0136+3905',b'egal',[],[],0,0
7,214,b' ',b' ',b'1FHL J0152.6+0148 ',b'2FGL J0152.6+0148 ',b'PMN J0152+0146 ',b' ',b' ',b' ',b' ',...,b'tKGVcC',156,b'TeV J0152+017',8,b'TeV J0152+0146',b'egal',[],[],0,0
8,241,b' ',b'1FGL J0205.6+6449 ',b'1FHL J0205.7+6448 ',b'2FGL J0205.8+6448 ',b'PSR J0205+6449 ',b' ',b' ',b' ',b' ',...,b'2I2TuY',252,b'TeV J0209+648',9,b'TeV J0205+6451',b'gal',[],[],0,0
9,273,b'0FGL J0220.9+3607',b'1FGL J0221.0+3555 ',b'1FHL J0221.1+3555 ',b'2FGL J0221.0+3555 ',b'B0218+357 ',b' ',b' ',b' ',b' ',...,b'hJnA37',254,b'TeV J0218+359',10,b'TeV J0221+3556',b'egal',[],[],0,0


In [25]:
catalog.head()

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad,marked
0,19,b'0FGL J0007.4+7303',b'1FGL J0007.0+7303 ',b'1FHL J0007.3+7303 ',b'2FGL J0007.0+7303 ',b'LAT PSR J0007+7303 ',b' ',b'1AGL J0006+7311',b' ',b'EGR J0008+7308 ',...,b'rC5JCj',227,b'TeV J0006+729',1,b'TeV J0006+7259',b'gal',[],[],0,0
1,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'70hRXZ',213,b'TeV J0013-188',2,b'TeV J0013-1853',b'egal',[],[],0,0
2,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'xCyAjF',209,b'TeV J0025+641',3,b'TeV J0025+6410',b'gal',[],[],0,0
3,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'PKJZqs',239,b'TeV J0033-193',4,b'TeV J0033-1921',b'egal',[],[],0,0
4,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'eNYfrm',229,b'TeV J0035+598',5,b'TeV J0035+5947',b'egal',[],[],0,0


Columns contained spectrum in gev:

In [26]:
gev_spectrum_columns = [
    'gev_nuFnu10000_100000',
    'gev_nuFnu1000_3000',
    'gev_nuFnu100_300',
    'gev_nuFnu3000_10000',
    'gev_nuFnu300_1000',
    'gev_nuFnu30_100',
]

Columns contained spectrum in simbad:

In [27]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [28]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [29]:
s_other_columns = ['s_MAIN_ID',
                 's_class', 
                 's_simbad'  
                 ]

In [30]:
gev_other_columns = ['gev_1FGL_Name',
                     'gev_CLASS1',
                    ]

In [31]:
tev_other_columns = ['tev_fermi_names', 
                     'tev_classes',
                     'tev_glon',
                     'tev_glat'
                    ]

In [32]:
spectrum_columns = gev_spectrum_columns + tev_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [33]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [34]:
#spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [35]:
#for i in tev_spectrum_columns:
#    if len(re.findall("\d+", i)) == 2:
#        E = 0.3
#    else:
#        E = int(re.findall("\d+", i)[0])
#    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [36]:
spectrums = spectrums.fillna(0)
#only_spectra = spectrums[spectrum_columns].copy()
#only_spectra = np.log(only_spectra)
#only_spectra[(only_spectra == -np.inf)] = 0
#spectrums[spectrum_columns] = only_spectra

In [37]:
spectrums

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,s_MAIN_ID,s_class,s_simbad,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,tev_glon,tev_glat,marked
0,1.341146e-11,1.174087e-10,4.279375e-11,9.882704e-11,8.130162e-11,0.0,1.330936e-12,2.014754e-12,1.128272e-12,1.532681e-12,...,[],[],0,b'1FGL J0007.0+7303 ',b'PSR ',b'',"b'pwn,snr'",119.580254,10.203682,0
1,1.474320e-13,5.385566e-13,1.167986e-17,6.283425e-13,7.551128e-14,0.0,1.046814e-12,1.947057e-13,3.526498e-14,5.421819e-15,...,[],[],0,b' ',b'bll ',b'',b'hbl',74.632011,-78.086937,0
2,1.018493e-12,1.078210e-12,5.019687e-14,5.647267e-13,2.145694e-12,0.0,1.975593e-12,3.010336e-13,9.506755e-14,8.749738e-13,...,[],[],0,b' ',b'snr ',b'',b'snr',120.092361,1.412930,0
3,8.395114e-12,4.176084e-12,2.267279e-12,6.302554e-12,2.296574e-12,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0033.5-1921 ',b'bll ',b'',b'hbl',94.174644,-81.216103,0
4,5.756689e-12,3.369683e-12,2.990093e-12,4.719991e-12,4.563475e-12,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0035.9+5951 ',b'bll ',b'',b'hbl',120.975426,-2.978120,0
5,5.418662e-13,1.216714e-12,2.193543e-12,1.207612e-12,1.687071e-12,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0047.3-2512 ',b'sbg ',b'',b'galaxy',97.363853,-87.964546,0
6,1.884848e-11,6.488241e-12,4.589447e-12,1.018232e-11,3.958297e-12,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0136.5+3905 ',b'bll ',b'',b'hbl',132.416458,-22.939846,0
7,2.012875e-12,1.517194e-12,1.012592e-12,7.859896e-13,7.043175e-13,0.0,2.950720e-12,7.895815e-13,5.337927e-13,3.486100e-13,...,[],[],0,b' ',b'bll ',b'',b'hbl',152.377487,-57.540936,0
8,3.200856e-13,1.137615e-11,1.702918e-11,4.202137e-12,1.534279e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0205.6+6449 ',b'PSR ',b'',b'pwn',130.717819,3.083848,0
9,3.653335e-12,7.641505e-12,1.661823e-11,6.361454e-12,1.138472e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,[],[],0,b'1FGL J0221.0+3555 ',b'FSRQ ',b'',b'blazar',142.601715,-23.486858,0


Normalization:

In [38]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [39]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,s_MAIN_ID,s_class,s_simbad,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,tev_glon,tev_glat,marked
0,0.036762,0.321829,0.117302,0.270895,0.222856,0.0,0.003648,0.005523,0.003093,4.201234e-03,...,[],[],0,b'1FGL J0007.0+7303 ',b'PSR ',b'',"b'pwn,snr'",119.580254,10.203682,0
1,0.055155,0.201477,0.000004,0.235066,0.028249,0.0,0.391619,0.072840,0.013193,2.028332e-03,...,[],[],0,b' ',b'bll ',b'',b'hbl',74.632011,-78.086937,0
2,0.069119,0.073172,0.003407,0.038325,0.145616,0.0,0.134072,0.020429,0.006452,5.937936e-02,...,[],[],0,b' ',b'snr ',b'',b'snr',120.092361,1.412930,0
3,0.358190,0.178179,0.096737,0.268908,0.097987,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0033.5-1921 ',b'bll ',b'',b'hbl',94.174644,-81.216103,0
4,0.269005,0.157462,0.139724,0.220561,0.213247,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0035.9+5951 ',b'bll ',b'',b'hbl',120.975426,-2.978120,0
5,0.079141,0.177705,0.320375,0.176376,0.246403,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0047.3-2512 ',b'sbg ',b'',b'galaxy',97.363853,-87.964546,0
6,0.427725,0.147237,0.104148,0.231066,0.089825,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0136.5+3905 ',b'bll ',b'',b'hbl',132.416458,-22.939846,0
7,0.184803,0.139294,0.092967,0.072162,0.064664,0.0,0.270907,0.072492,0.049008,3.200608e-02,...,[],[],0,b' ',b'bll ',b'',b'hbl',152.377487,-57.540936,0
8,0.006631,0.235676,0.352788,0.087054,0.317851,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0205.6+6449 ',b'PSR ',b'',b'pwn',130.717819,3.083848,0
9,0.080013,0.167359,0.363962,0.139325,0.249341,0.0,0.000000,0.000000,0.000000,0.000000e+00,...,[],[],0,b'1FGL J0221.0+3555 ',b'FSRQ ',b'',b'blazar',142.601715,-23.486858,0


In [40]:
spectrums.to_csv("markeddata/gevtev_simbadclasses_normalized_WITHOUTLOG.txt", index=False, sep='\t')

In [20]:
def calculate_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    #if len(s_spectrum_columns) > 0:
    #    spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])
    #for i in tev_spectrum_columns:
    #    if len(re.findall("\d+", i)) == 2:
    #        E = 0.3
    #    else:
    #        E = int(re.findall("\d+", i)[0])
    #    spectrums[i] = 1.6 * E ** 2 * spectrums[i]
    spectrums = spectrums.fillna(0)
    only_spectra = spectrums[spectrum_columns].copy()
    only_spectra = np.log(only_spectra)
    only_spectra[(only_spectra == -np.inf)] = 0
    spectrums[spectrum_columns] = only_spectra
    
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized.txt", index=False)

In [21]:
#calculate_spectra(catalog_name="gevtev", 
#                  s_spectrum_columns=[], 
#                  gev_spectrum_columns=gev_spectrum_columns, 
#                  tev_spectrum_columns=tev_spectrum_columns, 
#                  s_other_columns=[],
#                  gev_other_columns=gev_other_columns,
#                  tev_other_columns=tev_other_columns)
#calculate_spectra(catalog_name="gevtev_simbadclasses", 
#                  s_spectrum_columns=[], 
#                  gev_spectrum_columns=gev_spectrum_columns, 
#                  tev_spectrum_columns=tev_spectrum_columns, 
#                  s_other_columns=['s_class', 's_MAIN_ID', 's_simbad' ],
#                  gev_other_columns=gev_other_columns,
#                  tev_other_columns=tev_other_columns)
#calculate_spectra(catalog_name="tevsimbad", 
#                  s_spectrum_columns =s_spectrum_columns, 
#                  gev_spectrum_columns=[], 
#                  tev_spectrum_columns=tev_spectrum_columns, 
#                  s_other_columns=s_other_columns,
#                  gev_other_columns=[],
#                  tev_other_columns=tev_other_columns)
#calculate_spectra(catalog_name="gevsimbad", 
#                  s_spectrum_columns=s_spectrum_columns, 
#                  gev_spectrum_columns=gev_spectrum_columns, 
#                  tev_spectrum_columns=[], 
#                  s_other_columns=s_other_columns,
#                  gev_other_columns=gev_other_columns,
#                  tev_other_columns=[])